In [58]:
import cirq
import numpy as np 
from cirq import protocols, value
from typing import Dict, List, Optional, Sequence, Union, Collection, Tuple  


In [31]:
pip install monkeytype==19.5.0


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [36]:
from monkeytype import Optional 

In [29]:
    def execute(self, sql, params=None):

        def execute(self, sql: str, params: Optional[Union[Tuple[int], Tuple[str, str]]] = None):
             start = get_time()
                    try:

IndentationError: unexpected indent (<ipython-input-29-87dacb91de6a>, line 5)

In [59]:
from cirq._compat import proper_repr
#from cirq._doc import document
from cirq.ops import (controlled_gate, eigen_gate, gate_features, raw_types)

from cirq.type_workarounds import NotImplementedType


from cirq.ops.swap_gates import ISWAP, SWAP, ISwapPowGate, SwapPowGate
from cirq.ops.measurement_gate import MeasurementGate

In [60]:
# Get a qubit and a circuit
qbit = cirq.LineQubit(0)
circuit = cirq.Circuit()


# Add an X gate: acts like the Pauli Matrix sigma_x
circuit.append(cirq.X(qbit))


# Run a simple simulation that extracts the wavefunction of this state
sim = cirq.Simulator()
result = sim.simulate(circuit)

#print("\nBloch Sphere of the qubit in the final state:")
#state = cirq.bloch_vector_from_state_vector(result.final_state,0)
#print("x: ", around(state[0], 4), " y: ", around(state[1], 4),
#" z: ", around(state[2], 4))


# Add a measurement at the end of the circuit:
circuit.append(cirq.measure(qbit, key="Final state"))


# Display the circuit:
print("\nCircuit:")
print(circuit)


# Invoke the Cirq quantum simulator to execute the circuit:
simulator = cirq.Simulator()


# Simulate the circuit several times:
result = simulator.run(circuit, repetitions=10)


# Print the results:
print("\nResults of 10 trials:")
print(result)


Circuit:
0: ───X───M('Final state')───

Results of 10 trials:
Final state=1111111111


In [61]:
@value.value_equality
class XPowGate(eigen_gate.EigenGate,
               gate_features.SingleQubitGate):
    """A gate that rotates around the X axis of the Bloch sphere.

    The unitary matrix of ``XPowGate(exponent=t)`` is:

        [[g·c, -i·g·s],
         [-i·g·s, g·c]]

    where:

        c = cos(π·t/2)
        s = sin(π·t/2)
        g = exp(i·π·t/2).

    Note in particular that this gate has a global phase factor of
    e^{i·π·t/2} vs the traditionally defined rotation matrices
    about the Pauli X axis. See `cirq.rx` for rotations without the global
    phase. The global phase factor can be adjusted by using the `global_shift`
    parameter when initializing.

    `cirq.X`, the Pauli X gate, is an instance of this gate at exponent=1.
    """

    def _apply_unitary_(self, args: 'protocols.ApplyUnitaryArgs'
                       ) -> Optional[np.ndarray]:
        if self._exponent != 1:
            return NotImplemented
        zero = args.subspace_index(0)
        one = args.subspace_index(1)
        args.available_buffer[zero] = args.target_tensor[one]
        args.available_buffer[one] = args.target_tensor[zero]
        p = 1j**(2 * self._exponent * self._global_shift)
        if p != 1:
            args.available_buffer *= p
        return args.available_buffer

    def in_su2(self) -> 'XPowGate':
        """Returns an equal-up-global-phase gate from the group SU2."""
        return XPowGate(exponent=self._exponent, global_shift=-0.5)


    def with_canonical_global_phase(self) -> 'XPowGate':
        """Returns an equal-up-global-phase standardized form of the gate."""
        return XPowGate(exponent=self._exponent)


    def _eigen_components(self):
        return [
            (0, np.array([[0.5, 0.5], [0.5, 0.5]])),
            (1, np.array([[0.5, -0.5], [-0.5, 0.5]])),
        ]

    def _decompose_into_clifford_with_qubits_(self, qubits):
        from cirq.ops.clifford_gate import SingleQubitCliffordGate
        if self.exponent % 2 == 0:
            return []
        if self.exponent % 2 == 0.5:
            return SingleQubitCliffordGate.X_sqrt.on(*qubits)
        if self.exponent % 2 == 1:
            return SingleQubitCliffordGate.X.on(*qubits)
        if self.exponent % 2 == 1.5:
            return SingleQubitCliffordGate.X_nsqrt.on(*qubits)
        return NotImplemented

    def _trace_distance_bound_(self) -> Optional[float]:
        if self._is_parameterized_():
            return None
        return abs(np.sin(self._exponent * 0.5 * np.pi))

    def controlled(self,
                   num_controls: int = None,
                   control_values: Optional[Sequence[
                       Union[int, Collection[int]]]] = None,
                   control_qid_shape: Optional[Tuple[int, ...]] = None
                  ) -> raw_types.Gate:
        """
        Constructs CXPowGate from controlled XPowGate when applicable.

        This method is a specialized controlled method for XPowGate. It
        overrides the default behavior of returning a ControlledGate by
        transforming the underlying controlled gate to a CXPowGate and
        removing the last specified control qubit (which acts first
        semantically).  If this is a gate with multiple control qubits, it will
        now be a ControlledGate with one less control.

        This behavior only occurs when the last control qubit is a default-type
        control qubit. A default-type control qubit is one with shape of 2 (not
        a generic qudit) and where the control is satisfied by the qubit being
        ON, as opposed to OFF.

        (Note that a CXPowGate is, by definition, a controlled-XPowGate.)
        """
        result = super().controlled(num_controls, control_values,
                                    control_qid_shape)
        if (isinstance(result, controlled_gate.ControlledGate) and
                result.control_values[-1] == (1,) and
                result.control_qid_shape[-1] == 2):
            return cirq.CXPowGate(exponent=self._exponent,
                                  global_shift=self._global_shift).controlled(
                                      result.num_controls() - 1,
                                      result.control_values[:-1],
                                      result.control_qid_shape[:-1])
        return result


    def _pauli_expansion_(self) -> value.LinearDict[str]:
        if protocols.is_parameterized(self):
            return NotImplemented
        phase = 1j**(2 * self._exponent * (self._global_shift + 0.5))
        angle = np.pi * self._exponent / 2
        return value.LinearDict({
            'I': phase * np.cos(angle),
            'X': -1j * phase * np.sin(angle),
        })

    def _circuit_diagram_info_(self, args: 'cirq.CircuitDiagramInfoArgs'
                              ) -> Union[str, 'protocols.CircuitDiagramInfo']:
        if self._global_shift == -0.5:
            angle_str = self._format_exponent_as_angle(args)
            return f'Rx({angle_str})'

        return protocols.CircuitDiagramInfo(
            wire_symbols=('X',),
            exponent=self._diagram_exponent(args))

    def _qasm_(self, args: 'cirq.QasmArgs',
               qubits: Tuple['cirq.Qid', ...]) -> Optional[str]:
        args.validate_version('2.0')
        if self._exponent == 1:
            return args.format('x {0};\n', qubits[0])

        return args.format('rx({0:half_turns}) {1};\n', self._exponent,
                           qubits[0])

    @property
    def phase_exponent(self):
        return 0.0

    def _phase_by_(self, phase_turns, qubit_index):
        """See `cirq.SupportsPhase`."""
        return cirq.ops.phased_x_gate.PhasedXPowGate(
            exponent=self._exponent,
            phase_exponent=phase_turns * 2)

    def _has_stabilizer_effect_(self) -> Optional[bool]:
        if self._is_parameterized_():
            return None
        return self.exponent % 1 == 0

    def __str__(self) -> str:
        if self._global_shift == -0.5:
            if self._exponent == 1:
                return 'Rx(π)'
            return 'Rx({}π)'.format(self._exponent)
        if self._global_shift == 0:
            if self._exponent == 1:
                return 'X'
            return 'X**{}'.format(self._exponent)
        return ('XPowGate(exponent={}, '
                'global_shift={!r})').format(self._exponent, self._global_shift)

    def __repr__(self) -> str:
        if self._global_shift == -0.5:
            if protocols.is_parameterized(self._exponent):
                return 'cirq.rx({})'.format(
                    proper_repr(sympy.pi * self._exponent))

            return 'cirq.rx(np.pi*{})'.format(proper_repr(self._exponent))
        if self._global_shift == 0:
            if self._exponent == 1:
                return 'cirq.X'
            return '(cirq.X**{})'.format(proper_repr(self._exponent))
        return (
            'cirq.XPowGate(exponent={}, '
            'global_shift={!r})'
        ).format(proper_repr(self._exponent), self._global_shift)



In [62]:

class YPowGate(eigen_gate.EigenGate,
               gate_features.SingleQubitGate):
    def _apply_unitary_(self, args: 'protocols.ApplyUnitaryArgs'
                       ) -> Optional[np.ndarray]:
        if self._exponent != 1:
            return NotImplemented
        zero = args.subspace_index(0)
        one = args.subspace_index(1)
        args.available_buffer[zero] = -1j * args.target_tensor[one]
        args.available_buffer[one] = 1j * args.target_tensor[zero]
        p = 1j**(2 * self._exponent * self._global_shift)
        if p != 1:
            args.available_buffer *= p
        return args.available_buffer

    def in_su2(self) -> 'YPowGate':
        """Returns an equal-up-global-phase gate from the group SU2."""
        return YPowGate(exponent=self._exponent, global_shift=-0.5)


    def with_canonical_global_phase(self) -> 'YPowGate':
        """Returns an equal-up-global-phase standardized form of the gate."""
        return YPowGate(exponent=self._exponent)


    def _decompose_into_clifford_with_qubits_(self, qubits):
        from cirq.ops.clifford_gate import SingleQubitCliffordGate
        if self.exponent % 2 == 0:
            return []
        if self.exponent % 2 == 0.5:
            return SingleQubitCliffordGate.Y_sqrt.on(*qubits)
        if self.exponent % 2 == 1:
            return SingleQubitCliffordGate.Y.on(*qubits)
        if self.exponent % 2 == 1.5:
            return SingleQubitCliffordGate.Y_nsqrt.on(*qubits)
        return NotImplemented

    def _eigen_components(self):
        return [
            (0, np.array([[0.5, -0.5j], [0.5j, 0.5]])),
            (1, np.array([[0.5, 0.5j], [-0.5j, 0.5]])),
        ]

    def _trace_distance_bound_(self) -> Optional[float]:
        if self._is_parameterized_():
            return None
        return abs(np.sin(self._exponent * 0.5 * np.pi))

    def _pauli_expansion_(self) -> value.LinearDict[str]:
        if protocols.is_parameterized(self):
            return NotImplemented
        phase = 1j**(2 * self._exponent * (self._global_shift + 0.5))
        angle = np.pi * self._exponent / 2
        return value.LinearDict({
            'I': phase * np.cos(angle),
            'Y': -1j * phase * np.sin(angle),
        })

    def _circuit_diagram_info_(self, args: 'cirq.CircuitDiagramInfoArgs'
                              ) -> Union[str, 'protocols.CircuitDiagramInfo']:
        if self._global_shift == -0.5:
            angle_str = self._format_exponent_as_angle(args)
            return f'Ry({angle_str})'

        return protocols.CircuitDiagramInfo(
            wire_symbols=('Y',),
            exponent=self._diagram_exponent(args))

    def _qasm_(self, args: 'cirq.QasmArgs',
               qubits: Tuple['cirq.Qid', ...]) -> Optional[str]:
        args.validate_version('2.0')
        if self._exponent == 1:
            return args.format('y {0};\n', qubits[0])

        return args.format('ry({0:half_turns}) {1};\n', self._exponent,
                           qubits[0])

    @property
    def phase_exponent(self):
        return 0.5

    def _phase_by_(self, phase_turns, qubit_index):
        """See `cirq.SupportsPhase`."""
        return cirq.ops.phased_x_gate.PhasedXPowGate(
            exponent=self._exponent,
            phase_exponent=0.5 + phase_turns * 2)

    def _has_stabilizer_effect_(self) -> Optional[bool]:
        if self._is_parameterized_():
            return None
        return self.exponent % 1 == 0

    def __str__(self) -> str:
        if self._global_shift == -0.5:
            if self._exponent == 1:
                return 'Ry(π)'
            return 'Ry({}π)'.format(self._exponent)
        if self._global_shift == 0:
            if self._exponent == 1:
                return 'Y'
            return 'Y**{}'.format(self._exponent)
        return ('YPowGate(exponent={}, '
                'global_shift={!r})').format(self._exponent, self._global_shift)

    def __repr__(self) -> str:
        if self._global_shift == -0.5:
            if protocols.is_parameterized(self._exponent):
                return 'cirq.ry({})'.format(
                    proper_repr(sympy.pi * self._exponent))

            return 'cirq.ry(np.pi*{})'.format(proper_repr(self._exponent))
        if self._global_shift == 0:
            if self._exponent == 1:
                return 'cirq.Y'
            return '(cirq.Y**{})'.format(proper_repr(self._exponent))
        return (
            'cirq.YPowGate(exponent={}, '
            'global_shift={!r})'
        ).format(proper_repr(self._exponent), self._global_shift)


In [63]:
def ry(rads: value.TParamVal) -> YPowGate:
    """Returns a gate with the matrix e^{-i Y rads / 2}."""
    pi = sympy.pi if protocols.is_parameterized(rads) else np.pi
    return YPowGate(exponent=rads / pi, global_shift=-0.5)

In [65]:
def small_ansatz(parameter_y, qubit):
  #ygate = cirq.ry(parameter_y)
  ygate = cirq.YPowGate(exponent=parameter_y)
  #yield xgate(qubit)
  yield ygate(qubit)

#wrapping into a circuit

def psi_circuit(parameter):
  curr_q = cirq.LineQubit(0)
  curr_c = cirq.Circuit()

  curr_c.append(small_ansatz(parameter, curr_q))

  curr_c.append(cirq.measure(curr_q, key='q0'))
  return curr_c

def expectation_value_vqe(param, num_reps):
  curr_psi = psi_circuit(param)
  print()
  #keep the measured keys and values

  curr_simulator = cirq.Simulator()
  curr_results = curr_simulator.run(curr_psi, repetitions=num_reps) 
  s_k, s_v= zip(*r.measurements.items())

  #convert into booleans from sp|in values
  curr_state_values = 1 - 2*np.array(s_v).astype(np.int32)

  #get the expectation value (the average of the counts)
  #I keep double the variables because I want to remind myself of the distinction between this task and the context 
  #of Farhi's paper.
  curr_predicted_label_value = np.mean(curr_state_values)
  return(curr_predicted_label_value)

In [66]:
params_range = np.linspace(0.0, 2 * np.pi, 25)

data = [expectation_value_vqe(params,20) for params in params_range]

NameError: name 'r' is not defined

In [ ]:
params_range = np.linspace(0.0, 2 * np.pi, 20)
data = [expectation_value_vqe(params,20) for params in params_range]

import matplotlib.pyplot as plt
plt.xlabel('Angle [radians]')
plt.ylabel('Expectation value')
plt.plot(params_range, data)
plt.show()

In [ ]:
angle = 2.0
vqe_inst.expectation(small_ansatz([angle]), hamiltonian, 10000, qvm)

In [ ]:
angle_range = np.linspace(0.0, 2 * np.pi, 20)
data = [vqe_inst.expectation(small_ansatz([angle]), hamiltonian, None, qvm)
        for angle in angle_range]

import matplotlib.pyplot as plt
plt.xlabel('Angle [radians]')
plt.ylabel('Expectation value')
plt.plot(angle_range, data)
plt.show()